<a href="https://colab.research.google.com/github/Luzve/Credit_Card_Customers/blob/main/Credit__Card_Customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar librerías

In [ ]:
!pip install pyspark
!pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=d7b0636f104f85a0e1e0af9923809d4c6ff6220ad3fc08724c46dfeb8d8f2cb6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Importar librerías

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum
from pyspark.sql.types import DoubleType
from pyspark.sql.types import *
import zipfile
import os
import pandas as pd


# Creación de una sesión con Spark

In [ ]:
#Iniciar conexión de spark
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()


# Conexión con kaggle para descargar los datos
Para descargar datos desde Kaggle es necesario crear un token de acceso. Para esto debes seguir los siguientes pasos:

1. Ingresa a tu perfil de kaggle
2. Seleccionar Account y bajar hasta la opción API
3. Dar click en Create New API Token y se descargará un archivo JSON con tus credenciales
4. Subir el archivo JSON a Colab Notebook

In [ ]:
# Creamos carpeta oculta en ambiente de linux sobre colab
!mkdir ~/.kaggle

In [ ]:
# Copiar el archivo JSON  a la carpeta oculta que creamos
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
# Cambiamos los permisos para permitir lectura de las credenciales
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Descargamos archivos indicando el usuario del propietario de los datos en kaggle y el nombre de dataset
!kaggle datasets download sakshigoyal7/credit-card-customers --force

100% 379k/379k [00:00<00:00, 749kB/s]
100% 379k/379k [00:00<00:00, 749kB/s]


In [ ]:
# Descomprimir el archivo
for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()


# Lectura de datos

In [ ]:
df = spark.read.csv('/content/BankChurners.csv', header=True)

In [ ]:
!cp BankChurners.csv '/content/drive/MyDrive/datasets'

cp: cannot create regular file '/content/drive/MyDrive/datasets': No such file or directory


# Procesamiento de los datos

## Numero de fila y columnas

In [ ]:
print((df.count(), len(df.columns)))

(10127, 23)


# Ver muestra de los datos

In [ ]:
df.show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

# Esquema de los datos

In [ ]:
df.printSchema()

root
 |-- CLIENTNUM: string (nullable = true)
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: string (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: string (nullable = true)
 |-- Total_Relationship_Count: string (nullable = true)
 |-- Months_Inactive_12_mon: string (nullable = true)
 |-- Contacts_Count_12_mon: string (nullable = true)
 |-- Credit_Limit: string (nullable = true)
 |-- Total_Revolving_Bal: string (nullable = true)
 |-- Avg_Open_To_Buy: string (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: string (nullable = true)
 |-- Total_Trans_Amt: string (nullable = true)
 |-- Total_Trans_Ct: string (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: string (nullable = true)
 |-- Avg_Utilization_Ratio: string (nullable = 

# Estadística descriptiva

In [ ]:
df.describe().show()

+-------+-------------------+-----------------+-----------------+------+------------------+---------------+--------------+---------------+-------------+------------------+------------------------+----------------------+---------------------+-----------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|summary|          CLIENTNUM|   Attrition_Flag|     Customer_Age|Gender|   Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|    Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|     Credit_Limit|Total_Revolving_Bal|  Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|  Total_Trans_Amt|   To

# Filtrar el dataset

In [ ]:
df.filter(df.Gender == "F").show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

# Ordenar el dataset

In [ ]:
df.sort(F.col("Credit_Limit").desc()).show(10)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

# Eliminar una columna:

In [ ]:
df = df.drop("Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1")
df.show(10)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2|
+---------+-----------------

In [ ]:
df = df.drop("Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2")
df.show(10)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------

# Eliminar datos nulos:

In [ ]:
df = df.na.drop()
df.show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------

# Análisis de Gastos y Transacciones

In [ ]:
# Calcular el promedio de transacciones por cliente
promedio_transacciones = df.select(avg(col("Total_Trans_Ct"))).first()[0]
print("Promedio de transacciones por cliente:", promedio_transacciones)

Promedio de transacciones por cliente: 64.85869457884863


In [ ]:
# Calcular el gasto total por cliente
gasto_total = df.select(sum(col("Total_Trans_Amt"))).first()[0]
print("Gasto total de todos los clientes:", gasto_total)

Gasto total de todos los clientes: 44600182.0


In [ ]:
# Calcular el gasto total por categoría de ingresos
gasto_por_categoria = df.groupBy("Income_Category").agg(sum(col("Total_Trans_Amt")).alias("Total_Gasto"))
gasto_por_categoria.show()

+---------------+-----------+
|Income_Category|Total_Gasto|
+---------------+-----------+
|        $120K +|  3292923.0|
|    $60K - $80K|  6239899.0|
|   $80K - $120K|  6881344.0|
|        Unknown|  4761819.0|
|    $40K - $60K|  7886058.0|
| Less than $40K|1.5538139E7|
+---------------+-----------+



In [ ]:
promedio_transacciones = df.groupBy("CLIENTNUM").agg(avg(col("Total_Trans_Ct")).alias("Promedio_Transacciones"))
promedio_transacciones.show()

+---------+----------------------+
|CLIENTNUM|Promedio_Transacciones|
+---------+----------------------+
|716568708|                  27.0|
|824805858|                  49.0|
|708134283|                  31.0|
|721129383|                  42.0|
|714940608|                  53.0|
|771903033|                  40.0|
|815472108|                  31.0|
|812788233|                  43.0|
|709150608|                  64.0|
|710562258|                  47.0|
|718468158|                  37.0|
|716900433|                  61.0|
|781013958|                  37.0|
|719415858|                  71.0|
|720706008|                  69.0|
|787581858|                  59.0|
|756727233|                  66.0|
|716358033|                  65.0|
|715849983|                  82.0|
|709957158|                  65.0|
+---------+----------------------+
only showing top 20 rows



# Spark SQL en Python

## Creación de vista del dataframe

In [ ]:
df.createOrReplaceTempView("BankChurners")

## Muestra de los datos

In [ ]:
spark.sql("SELECT COUNT(*) FROM BankChurners").show()

+--------+
|count(1)|
+--------+
|   10127|
+--------+



 ## Valores únicos de una columna

In [ ]:
spark.sql("SELECT DISTINCT Customer_Age FROM BankChurners").show()

+------------+
|Customer_Age|
+------------+
|          51|
|          54|
|          29|
|          42|
|          73|
|          64|
|          30|
|          34|
|          59|
|          28|
|          35|
|          52|
|          47|
|          43|
|          31|
|          70|
|          61|
|          27|
|          26|
|          46|
+------------+
only showing top 20 rows



In [ ]:
spark.sql("SELECT AVG(Total_Trans_Ct) as avg_total_trans_ct FROM BankChurners").show()

+------------------+
|avg_total_trans_ct|
+------------------+
| 64.85869457884863|
+------------------+

